In [1]:
import pandas as pd
import numpy as np

# paths and filenames

In [2]:
path_orgtree = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 1 cleaning/cleaning step 2 - career/2.4 orgtree/"

In [3]:
filein_orgtree = "orgtree_4_2.xlsx"

In [4]:
fileout_orgtree = "orgtree_4_3_jeongsu.xlsx"

In [5]:
orgtree = pd.read_excel(path_orgtree+filein_orgtree)
orgtree.shape

(1290, 8)

In [6]:
orgtree.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'SupervisesParent', 'LinkToCurrent_PrimaryInstitution',
       'LinkToCurrent_OrgName', 'Notes'],
      dtype='object')

# clean Link fields

In [7]:
# replace NAN with Deprecated
orgtree[orgtree["LinkToCurrent_OrgName"]=="NAN"]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName,Notes
797,1.0,내각,수매양정성,NaN,NaN,NAN,NAN,NaN
806,1.0,내각,외무성_X국_대련무역대표부,NaN,NaN,NAN,NAN,NaN
808,1.0,내각,외무성_X국_몰타대사관,NaN,NaN,NAN,NAN,NaN
903,1.0,내각,일용품공업성,NaN,NaN,NAN,NAN,NaN
970,1.0,노동당,당중앙위원회_경제정책검열부,NaN,NaN,NAN,NAN,NaN
1004,1.0,노동당,당중앙위원회_신소실,중앙위원회,NaN,NAN,NAN,NaN
1086,1.0,정무원,원유공업부,NaN,NaN,NAN,NAN,NaN


In [8]:
orgtree.loc[orgtree["LinkToCurrent_PrimaryInstitution"]=="NAN","LinkToCurrent_PrimaryInstitution"] = "Deprecated"

In [9]:
orgtree.loc[orgtree["LinkToCurrent_OrgName"]=="NAN","LinkToCurrent_OrgName"] = "Deprecated"

In [10]:
orgtree[orgtree["LinkToCurrent_PrimaryInstitution"]=="NAN"]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName,Notes


In [11]:
orgtree[orgtree["LinkToCurrent_OrgName"]=="NAN"]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName,Notes


In [12]:
# replace LinkToCurrent_PrimaryInstitution==nan with "Please_Revise"

In [13]:
orgtree.loc[orgtree["LinkToCurrent_PrimaryInstitution"].isna(),"LinkToCurrent_PrimaryInstitution"] = "Please_Revise"

In [14]:
orgtree[orgtree["LinkToCurrent_PrimaryInstitution"].isna()]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName,Notes


In [15]:
# change and add link columns

In [16]:
orgtree["LinkToNext_Year"]=np.nan

In [17]:
orgtree.rename(columns={"LinkToCurrent_PrimaryInstitution":"LinkToNext_PI","LinkToCurrent_OrgName":"LinkToNext_Org"},inplace=True)
orgtree.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'SupervisesParent', 'LinkToNext_PI', 'LinkToNext_Org', 'Notes',
       'LinkToNext_Year'],
      dtype='object')

In [18]:
columns_orgtree = ['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'SupervisesParent', 'LinkToNext_PI', 'LinkToNext_Org','LinkToNext_Year','Notes']

In [19]:
orgtree = orgtree[columns_orgtree]
orgtree.shape

(1290, 9)

In [20]:
orgtree.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'SupervisesParent', 'LinkToNext_PI', 'LinkToNext_Org',
       'LinkToNext_Year', 'Notes'],
      dtype='object')

# strip white space from all fields

In [21]:
# be sure to strip white space from joborglink as well

In [22]:
orgtree["PrimaryInstitution"] = orgtree["PrimaryInstitution"].str.strip()

In [23]:
orgtree["OrgName"] = orgtree["OrgName"].str.strip()

In [24]:
orgtree["LinkToNext_PI"] = orgtree["LinkToNext_PI"].str.strip()

In [25]:
orgtree["LinkToNext_Org"] = orgtree["LinkToNext_Org"].str.strip()

# add LinkToNext_PI & LinkToNext_Org as Primary Key

In [26]:
orgtree_links = orgtree[['InsideGov','LinkToNext_PI', 'LinkToNext_Org']]

In [27]:
merged = orgtree.merge(orgtree_links,how="outer",
                        left_on=['PrimaryInstitution', 'OrgName'],
                        right_on=['LinkToNext_PI', 'LinkToNext_Org'],
                       indicator=True)

In [28]:
merged["_merge"].unique()

['left_only', 'both', 'right_only']
Categories (3, object): ['left_only', 'right_only', 'both']

In [29]:
merged[merged["_merge"]=="both"].shape

(245, 13)

In [30]:
merged[merged["_merge"]=="left_only"].shape

(1152, 13)

In [31]:
merged[merged["_merge"]=="right_only"].shape

(1045, 13)

In [32]:
merged_right = merged[merged["_merge"]=="right_only"]
merged_right.shape

(1045, 13)

In [33]:
merged_right.columns

Index(['InsideGov_x', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'SupervisesParent', 'LinkToNext_PI_x', 'LinkToNext_Org_x',
       'LinkToNext_Year', 'Notes', 'InsideGov_y', 'LinkToNext_PI_y',
       'LinkToNext_Org_y', '_merge'],
      dtype='object')

In [34]:
# 37 [LinkToNext_PI, LinkToNext_Org] not contained in [PI, Org]
merged_right = merged_right.drop_duplicates()
merged_right.shape

(35, 13)

In [35]:
merged_right

,InsideGov_x,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI_x,LinkToNext_Org_x,LinkToNext_Year,Notes,InsideGov_y,LinkToNext_PI_y,LinkToNext_Org_y,_merge
1397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Current,Current,right_only
1850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Current,Current,right_only
2366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,평양예술단,NaN,right_only
2367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Please_Revise,Please_Revise,right_only
2374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Please_Revise,Please_Revise,right_only
2376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,내각,교육위원회_고등교육성_계응상사리원농업대학,right_only
2377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Uncertain,Uncertain,right_only
2381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Uncertain,Uncertain,right_only
2398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Please_Revise,NaN,right_only
2400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Please_Revise,NaN,right_only


In [36]:
# prep merged_right columns for concat with orgtree
columns_select = ['InsideGov_y','LinkToNext_PI_y','LinkToNext_Org_y',
                  'ImmediateSuperiorOrg','SupervisesParent',
                  'LinkToNext_PI_x','LinkToNext_Org_x','LinkToNext_Year','Notes']

In [37]:
merged_right = merged_right[columns_select]
merged_right.shape

(35, 9)

In [38]:
merged_right.columns = orgtree.columns

In [39]:
merged_right

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes
1397,0.0,Current,Current,NaN,NaN,NaN,NaN,NaN,NaN
1850,1.0,Current,Current,NaN,NaN,NaN,NaN,NaN,NaN
2366,0.0,평양예술단,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2367,0.0,Please_Revise,Please_Revise,NaN,NaN,NaN,NaN,NaN,NaN
2374,1.0,Please_Revise,Please_Revise,NaN,NaN,NaN,NaN,NaN,NaN
2376,0.0,내각,교육위원회_고등교육성_계응상사리원농업대학,NaN,NaN,NaN,NaN,NaN,NaN
2377,0.0,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN
2381,1.0,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN
2398,0.0,Please_Revise,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2400,1.0,Please_Revise,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# concatenate orgtree & merged_right
orgtree = pd.concat([orgtree,merged_right])

In [41]:
orgtree.shape

(1325, 9)

In [42]:
# Step 1. drop NA keys
orgtree.dropna(subset=["PrimaryInstitution","OrgName"],how="all",inplace=True)
orgtree.shape

(1325, 9)

In [43]:
# Step 2. verify no duplicate rows
orgtree[orgtree.duplicated()]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes


In [44]:
# Step 2. verify no duplicate keys
orgtree.sort_values(["PrimaryInstitution","OrgName","InsideGov"],inplace=True)
orgtree[orgtree.duplicated(["PrimaryInstitution","OrgName"],keep=False)]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes
1397,0.0,Current,Current,NaN,NaN,NaN,NaN,NaN,NaN
1850,1.0,Current,Current,NaN,NaN,NaN,NaN,NaN,NaN
2420,1.0,Deprecated,Deprecated,NaN,NaN,NaN,NaN,NaN,NaN
2427,NaN,Deprecated,Deprecated,NaN,NaN,NaN,NaN,NaN,NaN
2367,0.0,Please_Revise,Please_Revise,NaN,NaN,NaN,NaN,NaN,NaN
2374,1.0,Please_Revise,Please_Revise,NaN,NaN,NaN,NaN,NaN,NaN
2398,0.0,Please_Revise,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2400,1.0,Please_Revise,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2409,NaN,Please_Revise,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2377,0.0,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# drop all of the above duplicates
orgtree.drop_duplicates(["PrimaryInstitution","OrgName"],keep=False,inplace=True)
orgtree.shape

(1314, 9)

# recursively add ImmedSuperiorOrgs, drop duplicates and sort

In [46]:
orgtree.sort_values(["InsideGov","PrimaryInstitution","OrgName"],ascending=[False,True,True],inplace=True)

In [47]:
orgtree.shape

(1314, 9)

In [48]:
orgtree[0:2]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes
537,1.0,NAN,인민군,NaN,NaN,Please_Revise,Please_Revise,NaN,NaN
539,1.0,국무위원회,국가보위성,NaN,NaN,Current,Current,NaN,NaN


In [49]:
# orgtree with non-NA OrgName
orgtree_proc = orgtree[~orgtree["OrgName"].isna()]
orgtree_proc.shape

(1044, 9)

In [50]:
orgtree_2order = orgtree_proc.loc[orgtree_proc.OrgName.str.contains("_"),["PrimaryInstitution","OrgName"]]
orgtree_2order["OrgName"] = orgtree_2order["OrgName"].apply(lambda x: "_".join(x.split("_")[:-1]))
orgtree_2order.drop_duplicates(inplace=True)
orgtree_2order.shape

(183, 2)

In [51]:
orgtree_2order[0:10]

,PrimaryInstitution,OrgName
540,국무위원회,국가보위성
542,국무위원회,국가체육지도위원회
2415,국무위원회,국방성
548,국무위원회,국방성_참모부
578,국무위원회,사회안전성
579,국무위원회,사회안전성_본부
592,국무위원회,사회안전성_참모부
620,국방위원회,국가안전보위부
623,국방위원회,사회안전부
625,국방위원회,사회안전성


In [52]:
orgtree_3order = orgtree_2order.loc[orgtree_2order.OrgName.str.contains("_"),["PrimaryInstitution","OrgName"]]
orgtree_3order["OrgName"] = orgtree_3order["OrgName"].apply(lambda x: "_".join(x.split("_")[:-1]))
orgtree_3order.drop_duplicates(inplace=True)
orgtree_3order.shape

(30, 2)

In [53]:
orgtree_3order[0:10]

,PrimaryInstitution,OrgName
548,국무위원회,국방성
579,국무위원회,사회안전성
698,내각,내무성
710,내각,농업위원회
2439,내각,대외경제성
726,내각,대외경제성_민족경제협력위원회
765,내각,문화성
806,내각,외무성
814,내각,외무성_국제기구국
899,내각,인민봉사총국


In [54]:
orgtree_4order = orgtree_3order.loc[orgtree_3order.OrgName.str.contains("_"),["PrimaryInstitution","OrgName"]]
orgtree_4order.OrgName = orgtree_4order.OrgName.apply(lambda x: "_".join(x.split("_")[:-1]))
orgtree_4order.drop_duplicates(inplace=True)
orgtree_4order.shape

(9, 2)

In [55]:
orgtree_5order = orgtree_4order.loc[orgtree_4order.OrgName.str.contains("_"),["PrimaryInstitution","OrgName"]]
orgtree_5order.OrgName = orgtree_5order.OrgName.apply(lambda x: "_".join(x.split("_")[:-1]))
orgtree_5order.drop_duplicates(inplace=True)
orgtree_5order.shape

(1, 2)

In [56]:
orgtree_5order

,PrimaryInstitution,OrgName
1131,총참모부,작전총국


In [57]:
# add all PrimaryInstitutions with null OrgName

In [58]:
orgtree_PI_nullOrgName = pd.DataFrame({"PrimaryInstitution":orgtree_proc.PrimaryInstitution.unique(),"OrgName":np.nan})
orgtree_PI_nullOrgName.shape

(77, 2)

In [97]:
orgtree_PI_nullOrgName

,PrimaryInstitution,OrgName
0,NAN,NaN
1,국무위원회,NaN
2,국방위원회,NaN
3,내각,NaN
4,노동당,NaN
...,...,...
72,체육연구원,NaN
73,태천발전소,NaN
74,평양만경대예술단,NaN
75,평양영화대학,NaN


In [98]:
orgtree.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'SupervisesParent', 'LinkToNext_PI', 'LinkToNext_Org',
       'LinkToNext_Year', 'Notes'],
      dtype='object')

In [99]:
orgtree_higher_order = pd.concat([orgtree_2order,orgtree_3order,orgtree_4order,orgtree_5order,orgtree_PI_nullOrgName])
orgtree_higher_order.drop_duplicates(inplace=True)
orgtree_higher_order.sort_values(["PrimaryInstitution","OrgName"],ascending=False,inplace=True)
orgtree_higher_order.shape

(264, 2)

In [61]:
orgtree_higher_order[orgtree_higher_order.duplicated(["PrimaryInstitution","OrgName"],keep=False)]

,PrimaryInstitution,OrgName


In [62]:
# find new orgtree_higher_order whose PrimaryOrg & OrgName are not contained in orgtree
orgtree_higher_order_merged = orgtree_higher_order.merge(orgtree[["PrimaryInstitution","OrgName"]],
                                             how="left",on=["PrimaryInstitution","OrgName"],
                                             suffixes=("","_right"),indicator=True)
orgtree_higher_order_unique = orgtree_higher_order_merged[orgtree_higher_order_merged["_merge"]=="left_only"]
orgtree_higher_order_unique.shape

(73, 3)

In [63]:
orgtree.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'SupervisesParent', 'LinkToNext_PI', 'LinkToNext_Org',
       'LinkToNext_Year', 'Notes'],
      dtype='object')

In [64]:
orgtree_higher_order_unique.columns

Index(['PrimaryInstitution', 'OrgName', '_merge'], dtype='object')

In [65]:
# drop _merge column in orgtree_higher_order_unique
orgtree_higher_order_unique = orgtree_higher_order_unique.drop(columns=["_merge"],axis=1)
orgtree_higher_order_unique.shape

(73, 2)

In [66]:
orgtree.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'SupervisesParent', 'LinkToNext_PI', 'LinkToNext_Org',
       'LinkToNext_Year', 'Notes'],
      dtype='object')

In [67]:
# add columns [LinkToNext_PI, LinkToNext_Org, Notes] and set them to null
orgtree_higher_order_unique["InsideGov"] = "Please_Revise"
orgtree_higher_order_unique["ImmediateSuperiorOrg"] = np.nan
orgtree_higher_order_unique["SupervisesParent"] = np.nan
orgtree_higher_order_unique["LinkToNext_PI"] = "Please_Revise"
orgtree_higher_order_unique["LinkToNext_Org"] = "Please_Revise"
orgtree_higher_order_unique["LinkToNext_Year"] = "Please_Revise"
orgtree_higher_order_unique["Notes"] = np.nan

In [68]:
orgtree_higher_order_unique = orgtree_higher_order_unique[orgtree.columns]

In [69]:
orgtree_higher_order_unique.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'SupervisesParent', 'LinkToNext_PI', 'LinkToNext_Org',
       'LinkToNext_Year', 'Notes'],
      dtype='object')

In [70]:
orgtree_higher_order_unique

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes
3,Please_Revise,평양영화대학,NaN,NaN,NaN,Please_Revise,Please_Revise,Please_Revise,NaN
4,Please_Revise,평양만경대예술단,NaN,NaN,NaN,Please_Revise,Please_Revise,Please_Revise,NaN
5,Please_Revise,태천발전소,NaN,NaN,NaN,Please_Revise,Please_Revise,Please_Revise,NaN
16,Please_Revise,총참모부,총참모부,NaN,NaN,Please_Revise,Please_Revise,Please_Revise,NaN
18,Please_Revise,총참모부,전략군사령부,NaN,NaN,Please_Revise,Please_Revise,Please_Revise,NaN
...,...,...,...,...,...,...,...,...,...
259,Please_Revise,개성방직공장,NaN,NaN,NaN,Please_Revise,Please_Revise,Please_Revise,NaN
260,Please_Revise,Please_Revise,NaN,NaN,NaN,Please_Revise,Please_Revise,Please_Revise,NaN
261,Please_Revise,NAN,NaN,NaN,NaN,Please_Revise,Please_Revise,Please_Revise,NaN
262,Please_Revise,6.15공동선언실천남북공동위원회북측위원회,NaN,NaN,NaN,Please_Revise,Please_Revise,Please_Revise,NaN


In [71]:
# concatenate orgtree and orgtree_higher_order_unique
orgtree_concat = pd.concat([orgtree,orgtree_higher_order_unique])
orgtree_concat.shape

(1387, 9)

In [72]:
# confirm no duplicate (PrimaryInstitution, OrgName) pairs 
orgtree_concat[orgtree_concat.duplicated(["PrimaryInstitution","OrgName"],keep=False)]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes


In [73]:
# add ImmediateSuperiorOrg to orgtree_higher_order_unique
orgtree_concat["ImmediateSuperiorOrg"] = orgtree_concat["OrgName"].apply(lambda row: "_".join(str(row).split("_")[:-1]))

In [104]:
# sort
orgtree_concat.sort_values(["PrimaryInstitution","OrgName"],ascending=True,inplace=True,ignore_index=True)

In [105]:
orgtree_concat.shape

(1387, 9)

In [106]:
orgtree_concat[0:5]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes
0,0.0,4.15문화창작단,NaN,,NaN,Current,Current,Current,NaN
1,0.0,6.15공동선언실천남북공동위원회북측위원회,교직원분과,,NaN,Current,Current,Current,NaN
2,0.0,6.15공동선언실천남북공동위원회북측위원회,노동자분과,,NaN,Current,Current,Current,NaN
3,0.0,6.15공동선언실천남북공동위원회북측위원회,농업근로자분과,,NaN,Current,Current,Current,NaN
4,0.0,6.15공동선언실천남북공동위원회북측위원회,문학예술분과,,NaN,Current,Current,Current,NaN


In [77]:
orgtree = orgtree_concat

In [107]:
orgtree

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes
0,0.0,4.15문화창작단,NaN,,NaN,Current,Current,Current,NaN
1,0.0,6.15공동선언실천남북공동위원회북측위원회,교직원분과,,NaN,Current,Current,Current,NaN
2,0.0,6.15공동선언실천남북공동위원회북측위원회,노동자분과,,NaN,Current,Current,Current,NaN
3,0.0,6.15공동선언실천남북공동위원회북측위원회,농업근로자분과,,NaN,Current,Current,Current,NaN
4,0.0,6.15공동선언실천남북공동위원회북측위원회,문학예술분과,,NaN,Current,Current,Current,NaN
...,...,...,...,...,...,...,...,...,...
1382,0.0,황해제철연합기업소,NaN,,NaN,Current,Current,Current,NaN
1383,0.0,후방군관학교,NaN,,NaN,강건종합군관학교,Please_Revise,Please_Revise,NaN
1384,0.0,희천정밀기계공장,NaN,,NaN,Current,Current,Current,NaN
1385,1.0,NaN,중앙방송위원회,,NaN,내각,문화성_조선중앙방송위원회,Please_Revise,NaN


# Please_Revise null InsideGov and LinkToNext_

In [78]:
orgtree.loc[orgtree["LinkToNext_PI"].isna()].shape

(24, 9)

In [79]:
orgtree.loc[orgtree["LinkToNext_PI"].isna(),"LinkToNext_PI"] = "Please_Revise"

In [80]:
orgtree.loc[orgtree["LinkToNext_Org"].isna()].shape

(83, 9)

In [81]:
orgtree.loc[orgtree["LinkToNext_Org"].isna(),"LinkToNext_Org"] = "Please_Revise"

In [82]:
orgtree.loc[orgtree["LinkToNext_Year"].isna()].shape

(1314, 9)

In [83]:
orgtree.loc[orgtree["LinkToNext_Year"].isna(),"LinkToNext_Year"] = "Please_Revise"

In [84]:
orgtree.loc[orgtree["LinkToNext_PI"]=="Current","LinkToNext_Year"] = "Current"

In [85]:
orgtree.loc[orgtree["InsideGov"].isna()].shape

(2, 9)

In [86]:
orgtree.loc[orgtree["InsideGov"].isna(),"InsideGov"] = "Please_Revise"

# double-check jol & orgtree

In [87]:
# confirmed all jol key are in orgtree!

# all exceptions are "Uncertain"

In [88]:
path_joborglink = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 1 cleaning/cleaning step 2 - career/2.3 joborglink/"

In [89]:
fileout_joborglink = "joborglink_7_2.xlsx"

In [90]:
jol = pd.read_excel(path_joborglink+fileout_joborglink)

In [91]:
merge = orgtree.merge(jol,how="outer",on=["PrimaryInstitution","OrgName"],indicator=True)

In [92]:
merge[merge["_merge"]=="both"].shape

(1416, 12)

In [93]:
merge[merge["_merge"]=="left_only"].shape

(693, 12)

In [94]:
merge[merge["_merge"]=="right_only"].shape

(10, 12)

In [95]:
merge[merge["_merge"]=="right_only"]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes_x,OrgString,Notes_y,_merge
2109,NaN,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,노동당 기계공업협회,기계공업부; 비고: 당중앙위원회?,right_only
2110,NaN,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,노동당 중공업부,중앙위원회->중앙인민위원회; 비고: 중앙인민위원회 is a PrimaryInstit...,right_only
2111,NaN,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,노동당 중앙인민위원회 중공업부 제1부,1부 Not found; 비고: 중앙인민위원회 is a PrimaryInstitut...,right_only
2112,NaN,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,당 기계공업부,NaN,right_only
2113,NaN,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,당 중공업부,NaN,right_only
2114,NaN,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,전자자동화공업위원회,NaN,right_only
2115,NaN,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,전자자동화공업위원회 기술국,기술국 Not found; 비고: PrimaryInstitution?,right_only
2116,NaN,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,전자자동화공업위원회 제1부,1부 Not found; 비고: PrimaryInstitution?,right_only
2117,NaN,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,제2집단군,NaN,right_only
2118,NaN,노동당,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,당 연락부,"Check the year. 문화연락부, 대남연락부, 대외연락부",right_only


# export orgtree

In [96]:
# orgtree.to_excel(path_orgtree+fileout_orgtree,index=False)